In [2]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re

if os.getlogin()=="JVARGH7":
   path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
   path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif os.getlogin()=='aamnasoniwala':
   path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
   path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

# path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
# path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Test Data Splits/Test Part "
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/clean_input.py")
execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")
execfile(path_equity_precision_llm_repo + "/functions/standardize_population.py")
execfile(path_equity_precision_llm_repo + "/functions/adjusted_source_population_match.py")

In [6]:
merged_df_test = pd.read_csv(f"{path_equity_precision_llm_folder}/llm training/epl04_merged_test_data.csv", na_values=['n/a','NaN'], keep_default_na=False)

merged_df_test.shape

(2076, 17)

In [9]:
# Split 'orig_source_population' on ',' and explode into long format
merged_df_test_long = merged_df_test.assign(
    orig_source_population=merged_df_test['orig_source_population'].str.split(',')
).explode('orig_source_population')

# Remove leading/trailing whitespace from the split values
merged_df_test_long['orig_source_population'] = merged_df_test_long['orig_source_population'].str.strip()

merged_df_test_long.reset_index(drop=True, inplace=True)

merged_df_test_long.shape
merged_df_test_long.to_csv(path_equity_precision_llm_folder + '/working/eplan01_merged_df_test_long.csv', index=False)

In [11]:
# Crosstab summaries for individual conditions, grouped by orig_source_population
summary_precision_medicine = merged_df_test_long.groupby('orig_source_population').apply(
    lambda g: crosstab_summary(g, truth='orig_precision_medicine', test='gpt_precision_medicine')
).reset_index(level=0).rename(columns={'orig_source_population': 'group'})

summary_diabetes = merged_df_test_long.groupby('orig_source_population').apply(
    lambda g: crosstab_summary(g, truth='orig_diabetes', test='gpt_diabetes')
).reset_index(level=0).rename(columns={'orig_source_population': 'group'})

summary_primary_study = merged_df_test_long.groupby('orig_source_population').apply(
    lambda g: crosstab_summary(g, truth='orig_primary_study', test='gpt_primary_study')
).reset_index(level=0).rename(columns={'orig_source_population': 'group'})

summary_precision_medicine['variable'] = 'Precision Medicine'
summary_diabetes['variable'] = 'Diabetes'
summary_primary_study['variable'] = 'Primary Study'

# Source population accuracy grouped by orig_source_population
def source_population_accuracy(g):
    t = pd.crosstab(g['source_population_match'], g['orig_source_population'])
    correct = t.loc[True].sum() if True in t.index else 0
    total = t.sum().sum()
    return correct / total if total > 0 else np.nan

summary_source_population = merged_df_test_long.groupby('orig_source_population').apply(
    source_population_accuracy
).reset_index().rename(columns={0: 'Accuracy', 'orig_source_population': 'group'})
summary_source_population['variable'] = 'Source Population'

# Overall total accuracy grouped by orig_source_population
def overall_accuracy(g):
    all_correct = (
        (g['orig_precision_medicine'] == g['gpt_precision_medicine']) &
        (g['orig_diabetes'] == g['gpt_diabetes']) &
        (g['orig_primary_study'] == g['gpt_primary_study'])
    )
    return all_correct.mean()

summary_overall_conditions = merged_df_test_long.groupby('orig_source_population').apply(
    overall_accuracy
).reset_index().rename(columns={0: 'Accuracy', 'orig_source_population': 'group'})
summary_overall_conditions['variable'] = 'Overall Conditions (Precision Medicine, Diabetes, Primary Study)'

# Combine all summaries into one df
df_summary = pd.concat([
    summary_precision_medicine,
    summary_diabetes,
    summary_primary_study,
    summary_source_population,
    summary_overall_conditions
], ignore_index=True)

df_summary


C:\Users\jvargh7\AppData\Local\Temp\ipykernel_38684\2487999634.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_precision_medicine = merged_df_test_long.groupby('orig_source_population').apply(
C:\Users\jvargh7\AppData\Local\Temp\ipykernel_38684\2487999634.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_diabetes = merged_df_test_long.groupby('orig_source_population').apply(
C:\Users\j

,group,Sensitivity,Specificity,Accuracy,ppv,npv,variable
0,Unknown,0.196970,0.869333,0.768707,0.209677,0.860158,Precision Medicine
1,ca,0.846154,0.951220,0.909091,0.846154,0.951220,Precision Medicine
2,cee,0.935484,0.933333,0.934066,0.878788,0.965517,Precision Medicine
3,ea,0.904762,0.764706,0.852861,0.867220,0.825397,Precision Medicine
4,lac,0.924242,0.786325,0.881890,0.907063,0.821429,Precision Medicine
5,mena,0.962963,1.000000,0.987500,1.000000,0.981481,Precision Medicine
6,na,0.930435,0.898876,0.916667,0.922414,0.909091,Precision Medicine
7,sa,0.914798,0.853933,0.897436,0.940092,0.800000,Precision Medicine
8,seap,0.888889,0.930233,0.914286,0.888889,0.930233,Precision Medicine
9,ssa,1.000000,0.857143,0.928571,0.875000,1.000000,Precision Medicine


In [16]:
df_summary_rounded = df_summary.copy()

# Convert numeric columns to percentages and string type, replace NaN with "-"
for col in numeric_cols:
    df_summary_rounded[col] = (df_summary_rounded[col] * 100).round(1).astype('str')
    df_summary_rounded[col] = df_summary_rounded[col].replace('nan', '-')
    variable_order = [
        'Precision Medicine',
        'Diabetes',
        'Primary Study',
        'Overall Conditions (Precision Medicine, Diabetes, Primary Study)',
        'Source Population'
    ]
    df_summary_rounded['variable'] = pd.Categorical(df_summary_rounded['variable'], categories=variable_order, ordered=True)
    df_summary_rounded = df_summary_rounded.sort_values(['group', 'variable'])


df_summary_rounded.to_csv(f"{path_equity_precision_llm_repo}/analysis/eplan01_checking for differential misclassification.csv", index=False)